In [18]:
import pandas as pd
import os

data_path = "D:\Coding\Resume-projects\\recommender-systems\\archive"

In [19]:
print("Loading dataframes")
try:
    orders = pd.read_csv(os.path.join(data_path, 'orders.csv'))
    products = pd.read_csv(os.path.join(data_path, 'products.csv'))
    order_products_prior = pd.read_csv(os.path.join(data_path, 'order_products__prior.csv'))
    order_products_train = pd.read_csv(os.path.join(data_path, 'order_products__train.csv'))
    aisles = pd.read_csv(os.path.join(data_path, 'aisles.csv'))
    departments = pd.read_csv(os.path.join(data_path, 'departments.csv'))
    print("Data loaded successfully!")
except FileNotFoundError:
    print(f"Error: make sure data_path is correct, {data_path}")

Loading dataframes
Data loaded successfully!


In [20]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [21]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [26]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [27]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [28]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [29]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [36]:
for name, df in {
    "orders": orders,
    "products": products,
    "order_products_prior": order_products_prior,
    "order_products_train": order_products_train,
    "aisles": aisles,
    "departments": departments
}.items():
    print(f"=== {name} ===")
    print(df.info())
    print(df.describe(), "\n\n")


=== orders ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB
None
           order_id       user_id  order_number     order_dow  \
count  3.421083e+06  3.421083e+06  3.421083e+06  3.421083e+06   
mean   1.710542e+06  1.029782e+05  1.715486e+01  2.776219e+00   
std    9.875817e+05  5.953372e+04  1.773316e+01  2.046829e+00   
min    1.000000e+00  1.000000e+00  1.000000e+00  0.000000e+00   
25%    8.552715e+05  5.139400e+04  5.000000e+00  1.000000e+00   
50%    1.710542e+06  1.026890e+05  1.100000e+01  3.000000e+00   
75%    2.565812e

In [37]:
orders["eval_set"].unique()

array(['prior', 'train', 'test'], dtype=object)

In [40]:
# Merge orders with prior order products on 'order_id' -> Link users with their prior orders
merged_df = pd.merge(orders, order_products_prior, on="order_id")
merged_df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
...,...,...,...,...,...,...,...,...,...,...
32434484,2977660,206209,prior,13,1,12,7.0,14197,5,1
32434485,2977660,206209,prior,13,1,12,7.0,38730,6,0
32434486,2977660,206209,prior,13,1,12,7.0,31477,7,0
32434487,2977660,206209,prior,13,1,12,7.0,6567,8,0


In [42]:
# Merge the result dataframe with products on 'product_id' -> link orders with product details
merged_df = pd.merge(merged_df, products, on='product_id')